In [1]:
import implicit
import lightfm
import pandas as pd 
import numpy as np
from implicit.bpr import BayesianPersonalizedRanking as BPR
from scipy import sparse
import orion_recommend
from orion_recommend.evaluate import metrics
import lightfm

In [31]:
train_np = np.load("new_fa_train_data.npy")
test_np = np.load("new_fa_test_data.npy")

In [3]:
ui_matrix = np.load("fa_ui_matrix.npy")

In [4]:
ua_matrix = np.load("new_fa_ua_matrix.npy")

In [5]:
from lightfm import cross_validation

In [6]:
train, test = cross_validation.random_train_test_split(sparse.coo_matrix(ui_matrix), test_percentage=0.2, random_state=None)

In [7]:
user_features = np.load("user_attributes_npc.npy")

In [8]:
user_features.shape

(86297, 60)

In [9]:
user_features = sparse.coo_matrix(user_features)

In [10]:
user_features.shape

(86297, 60)

In [11]:
item_features = np.load("new_fa_ia_matrix.npy")

In [12]:
item_features = sparse.coo_matrix(item_features)

In [13]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k


# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(train, user_features = user_features,
          item_features=item_features, epochs=100)

In [14]:
mf = BPR()

In [15]:
mf.fit(train)

  0%|          | 0/100 [00:00<?, ?it/s]

In [21]:
mf1 = LightFM(loss='bpr')
mf1.fit(train, epochs=100)

In [ ]:
rec_dic = {}
n=10
for u in user_list:
    user_preds = np.empty(shape=ui_matrix.shape[1])
    for i in range(ui_matrix.shape[1]):
        user_preds[i] = (mf1.predict(np.array([u]),np.array([i])))
    top_n = np.argsort(user_preds)[-n:]
    rec_dic[u] = top_n 

In [ ]:
def metrics_user_rec(item, sim_users, k):
    count = 0
    for i, user_list in zip(item, sim_users):       
        for u in user_list:
            if ui_matrix[u,i] == 1:
                count = count + 1            
    p_at_k = round(count/(item.shape[0] * k), 4)

    RS = []
    ans = 0.0
    for i, user_list in zip(item, sim_users):           
        r=[]
        for u in user_list:
             r.append(ui_matrix[u][i])
        ans = ans + evall.ndcg_at_k(r, k, method=1)
        RS.append(r)
    G_at_k = ans/item.shape[0]
    M_at_k = evall.mean_average_precision(RS)
    return p_at_k, G_at_k, M_at_k

def metrics_item_rec(user, sim_items, k):
    count = 0
    for u, item_list in zip(user, sim_items):       
        for i in item_list:
            if ui_matrix[u,i] == 1:
                count = count + 1            
    p_at_k = round(count/(user.shape[0] * k), 4)

    RS = []
    ans = 0.0
    for u, item_list in zip(user, sim_items):           
        r=[]
        for i in item_list:
             r.append(ui_matrix[u][i])
        ans = ans + evall.ndcg_at_k(r, k, method=1)
        RS.append(r)
    G_at_k = ans/user.shape[0]
    M_at_k = evall.mean_average_precision(RS)
    return p_at_k, G_at_k, M_at_k


In [24]:
# Evaluate the trained model
test_precision = precision_at_k(mf1, test,  k=10)

print(test_precision.mean())

train_precision = precision_at_k(mf1, train,  k=10)

print(train_precision.mean())

print(recall_at_k(mf1,test,  k=10).mean())

0.00010805554
0.072156735
0.001026527635204495


In [97]:
# Evaluate the trained model
test_precision = precision_at_k(model, test, user_features=item_features,
          item_features=user_features,  k=10)

test_precision.mean()

In [99]:
train_precision = precision_at_k(model, train, user_features=item_features,
          item_features=user_features,  k=10)

train_precision.mean()

In [100]:
recall_at_k(model,test, user_features=item_features,
          item_features = user_features,  k=10).mean()

3.1094527363184084e-05